VIC Pricing Model - This notebook is for pricing life settlements.

Import and define functions

In [486]:
import pandas as PD
import numpy as NP
import time as TM
# import matplotlib.pyplot as PLT
# import seaborn as SB
# import scipy.stats as ST
# import sklearn.model_selection as MS
# import sklearn.preprocessing as PR
# import sklearn.linear_model as LM

START_TIME = TM.perf_counter()

def YEARS_BETWEEN(early_date, later_date):
    return PD.Timestamp(later_date).year - PD.Timestamp(early_date).year - ((PD.Timestamp(later_date).month, PD.Timestamp(later_date).day) < (PD.Timestamp(early_date).month, PD.Timestamp(early_date).day))

def MONTHS_BETWEEN(date1, date2):
    dt1 = PD.Timestamp(date1)
    dt2 = PD.Timestamp(date2)
    return (dt2.year - dt1.year) * 12 + dt2.month - dt1.month - (dt2.day < dt1.day)

def ADD_YEARS(date, years):
    dt = PD.Timestamp(date)
    try:
        return dt.replace(year = dt.year + years)
    except ValueError:
        return dt.replace(month = 2, day = 28, year = dt.year + years)

def AGE_AT(date_of_birth, reference_date):
    return YEARS_BETWEEN(date_of_birth, reference_date)

def IS_PREMIUM_DUE(issue_date, current_date, mode):
    issue_ts = PD.Timestamp(issue_date)
    current_ts = PD.Timestamp(current_date)
    
    # Map mode to months
    mode_to_months = {
        'Annual': 12,
        'Semi-Annual': 6,
        'Quarterly': 3,
        'Monthly': 1
    }
    period_months = mode_to_months.get(mode, 12)  # Default to annual
    
    # Calculate months elapsed
    months_elapsed = (current_ts.year - issue_ts.year) * 12 + (current_ts.month - issue_ts.month)
    
    # Check if due (and day matches or is end of month)
    if months_elapsed % period_months == 0:
        # For monthly modes, due on the issue day or last day of month
        if current_ts.day == issue_ts.day or (current_ts.day == current_ts.days_in_month and issue_ts.day > current_ts.day):
            return 1
    return 0

def ROUND_SETTLEMENT(amount):
    if amount < 1000:
        increment = 50
    elif amount < 10000:
        increment = 500
    elif amount < 100000:
        increment = 5000
    else:
        increment = 50000
    
    return round(amount / increment, ndigits = 0) * increment

Initialize pricing inputs - NEED TO MOVE TO JSON INPUT

In [487]:
PRICING_ASSUMPTIONS = PD.read_csv("Data/Pricing_Assumptions.csv")

HURDLE_RATE = PRICING_ASSUMPTIONS.loc[0, "Hurdle_Rate"]                         # annual rate
HURDLE_RATE_MTHLY = (1 + HURDLE_RATE) ** (1/12) - 1                             # monthly rate

print(HURDLE_RATE, HURDLE_RATE_MTHLY)

# GROSS_PURCHASE_PRICE = 1000000                                                # dollars
UW_SPOT_ADJ = 1.0                                                               # multiplier
UW_WEAR_OFF_DUR = 25                                                            # years

0.5 0.03436608313191658


Initialize policy inputs - NEED TO MOVE TO JSON INPUT

In [488]:
DATE_OF_BIRTH = '1981-06-26'                # YYYY-MM-DD
ISSUED_FACE_AMOUNT = 10000000                # dollars
GENDER = 'Other'                            # 'M', 'F' or 'Other'
ISSUE_DATE = '2024-04-05'                   # YYYY-MM-DD
PRODUCT_TYPE = 'Term'                       # 'Term', 'WL', etc.
PRODUCT_DURATION = 20                       # years
PREMIUM_MODE = 'Semi-Annual'                     # 'Annual', 'Monthly', etc.
ISSUE_STATE = 'CA'                          # two-letter state code
MODAL_PREMIUM = 60                        # dollars
CLOSING_DATE = '2026-06-25'                 # YYYY-MM-DD

Starting calcs

In [489]:
ISSUED_DURATION = YEARS_BETWEEN(ISSUE_DATE, CLOSING_DATE)
print(ISSUED_DURATION)

if PRODUCT_DURATION == "Lifetime": 
  PRODUCT_DURATION_CALC = 121 - AGE_AT(ISSUE_DATE, DATE_OF_BIRTH)
else:
    PRODUCT_DURATION_CALC = PRODUCT_DURATION

MATURITY_DATE = ADD_YEARS(ISSUE_DATE, PRODUCT_DURATION_CALC)
ISSUE_AGE = AGE_AT(DATE_OF_BIRTH, ISSUE_DATE)
MORTALITY_TBL_BASE = "VBT15_" 

if GENDER == "M":
    MORTALITY_TBL_BASE = MORTALITY_TBL_BASE + "MNS_ALB"
else:
    MORTALITY_TBL_BASE = MORTALITY_TBL_BASE + "FNS_ALB"

MORTALITY_TBL_FILENAME = MORTALITY_TBL_BASE + ".csv"

ISSUE_DATE_DAY = PD.Timestamp(ISSUE_DATE).day
CLOSING_DATE_DAY = PD.Timestamp(CLOSING_DATE).day
print(ISSUE_DATE_DAY, CLOSING_DATE_DAY)

2
5 25


Load mortality table

In [490]:
MORTALITY_TBL = PD.read_csv("Data/" + MORTALITY_TBL_FILENAME)
MORTALITY_TBL.set_index('Iss_Age', inplace = True)              # Set index to 'Iss_Age' for proper lookups
# print(MORTALITY_TBL.head())

def MORTALITY_RATE(issue_age, duration):
    if issue_age + duration >= 121:
        return 0
    else:
        return MORTALITY_TBL.loc[issue_age + max(0, duration - 26), str(min(26, duration))]       
    
# mortality_rate_example = print(MORTALITY_RATE(19, 5))
# mortality_rate_example = print(MORTALITY_RATE(19, 25))
# mortality_rate_example = print(MORTALITY_RATE(19, 26))
# mortality_rate_example = print(MORTALITY_RATE(19, 27))

Initialize Pricing Model Data Frame

In [491]:
# Initialize Pricing Model Data Frame

# Calculate total months for the projection
TOTAL_MONTHS = PRODUCT_DURATION_CALC * 12

# Calculate starting policy year
POLICY_YEAR_START = ISSUED_DURATION + 1

# Generate lists for each column
LS_MONTH = list(range(1, TOTAL_MONTHS + 1))  # 1 to 240
# POLICY_MONTH = LS_MONTH  # Assuming same as life settlement month for now

# Generate dates starting from closing date
CLOSING_DATE_TS = PD.Timestamp(CLOSING_DATE)
FIRST_MONTHIVERSARY = CLOSING_DATE_TS.replace(day = PD.Timestamp(ISSUE_DATE).day) + PD.DateOffset(months = 1 * (CLOSING_DATE_DAY >= ISSUE_DATE_DAY))  # Next monthiversary after closing
# print(FIRST_MONTHIVERSARY)

END_DATES = [FIRST_MONTHIVERSARY + PD.DateOffset(months = i) + PD.DateOffset(days = -1) for i in range(TOTAL_MONTHS)]
# print(END_DATES[:12])  # Print first 12 end dates as a sample

BEG_DATES = [CLOSING_DATE_TS] + [END_DATES[i-1] + PD.DateOffset(days = 1) for i in range(1, TOTAL_MONTHS)]  # First is closing, then next day after each previous end date
# print(BEG_DATES[:12])  # Print first 12 beginning dates as a sample

# Calculate Policy_Duration and Mortality_Rate outside the DataFrame
POLICY_DURATIONS = [YEARS_BETWEEN(ISSUE_DATE, END_DATES[i]) for i in range(TOTAL_MONTHS)]
MORTALITY_RATES = [MORTALITY_RATE(ISSUE_AGE, POLICY_DURATIONS[i]) / 1000 for i in range(TOTAL_MONTHS)]


Add UW Adjustments to Mortality -- NEED TO ADD

In [492]:

# Calculate Qx and Px
QXS = [MORTALITY_RATES[i] * ((END_DATES[i] - BEG_DATES[i]).days + 1) / 365.25 for i in range(TOTAL_MONTHS)]
PXS = [1 - q for q in QXS]

# Calculate Sx recursively: Sx[i] = Sx[i-1] * Px[i-1], with Sx[0] = 1
SXS = [1.0]
for px in PXS:
    SXS.append(SXS[-1] * px)
SXS = SXS[1:]               # SXS now has survival at end of each month
SXS = [1.0] + SXS[:-1]      # Survival to beginning of month: shift by one, with 1.0 at start

MORTALITY_COSTS = [s * q * ISSUED_FACE_AMOUNT for s, q in zip(SXS, QXS)]
NPV_MORTALITY_COSTS = sum([cost / (1 + HURDLE_RATE_MTHLY) ** LS_MONTH[i - 1] for i, cost in enumerate(MORTALITY_COSTS)])

MODAL_PREMIUMS_DUE = [IS_PREMIUM_DUE(ISSUE_DATE, BEG_DATES[i], PREMIUM_MODE) * 1 for i in range(TOTAL_MONTHS)]
MODAL_PREMIUMS_TO_PAY = [p * MODAL_PREMIUM for p in MODAL_PREMIUMS_DUE]

NPV_MODAL_PREMIUMS_TO_PAY = sum([premium / (1 + HURDLE_RATE_MTHLY) ** LS_MONTH[i - 1] for i, premium in enumerate(MODAL_PREMIUMS_TO_PAY)])

# PRICING_MODEL_DF = PD.DataFrame({
#     'LS_Month': LS_MONTH,
#     'Policy_Duration': POLICY_DURATIONS,
#     'Beg_Date': BEG_DATES,
#     'End_Date': END_DATES,
#     'Days_In_Month':  [ (END_DATES[i] - BEG_DATES[i]).days + 1 for i in range(TOTAL_MONTHS) ],
#     'Sx': SXS,
#     'Qx': QXS,
#     'Px': PXS,
#     'Mortality_Costs': MORTALITY_COSTS,
#     'NPV_Mortality_Costs': NPV_MORTALITY_COSTS,
#     'Modal_Premium_Due': MODAL_PREMIUMS_DUE,
#     'Modal_Premium_To_Pay': MODAL_PREMIUMS_TO_PAY,
#     'NPV_Modal_Premiums_To_Pay': NPV_MODAL_PREMIUMS_TO_PAY
# })

# # Set pandas display options for wider output
# PD.set_option('display.width', 1200)
# PD.set_option('display.max_columns', None)

# print(PRICING_MODEL_DF.head(25))  # Print first 12 rows of the DataFrame


Total elapsed time

Calculate Settlement Amount

In [493]:

INIT_SETTLEMENT_AMOUNT = round(NPV_MORTALITY_COSTS - NPV_MODAL_PREMIUMS_TO_PAY, ndigits = 2)
SETTLEMENT_AMOUNT = ROUND_SETTLEMENT(INIT_SETTLEMENT_AMOUNT)

print("Initial Settlement Amount: $" + str(INIT_SETTLEMENT_AMOUNT))
print("Settlement Amount: $" + str(int(SETTLEMENT_AMOUNT)))

Initial Settlement Amount: $11647.83
Settlement Amount: $10000


In [494]:
END_TIME = TM.perf_counter()
print("Execution Time: " + str(round(END_TIME - START_TIME, ndigits = 1)) + "s")


Execution Time: 0.1s
